In [83]:
from playwright.async_api import async_playwright, Playwright, Page
from abc import ABC, abstractmethod

async def start_playwright():
    return await async_playwright().start()

async def get_chromium(playwright:Playwright, options:dict=None):
    if options is None:
        options = dict(
            headless=False,
            # headless=True,
            slow_mo=500,
            # args=[
            #     "--start-maximized",
            # ],
        )
    chromium = playwright.chromium
    return await chromium.launch(**options)


async def load_default_page(*, reset_session=False):
    playwright = await start_playwright()
    chrome = await get_chromium(playwright)
    context = None
    try:
        if reset_session:
            context = await chrome.new_context()
        else:
            context = await chrome.new_context(storage_state=".auth/state.json")
    except:
        context = await chrome.new_context()
    page = await context.new_page()
    await page.set_viewport_size({"width": 950, "height": 1000})
    page.set_default_timeout(6000)
    return page


##########################################
#
#          BasePage
#
##########################################
class BasePage(ABC):
    def __init__(self, page:Page):
        self.__page = page
        self.components()
    
    @property
    def page(self):
        return self.__page

    async def goto(self):
        await self.__page.goto(self.url())

    async def wait_for_no_overlay(self):
        overlay = self.page.locator('.blockUI.blockOverlay')
        await overlay.wait_for(state="detached", timeout=5000)
    
    @abstractmethod
    def url(self):
        raise ReferenceError("the url is empty")

    @abstractmethod
    def components(self):
        raise ReferenceError("should override components()")

In [126]:
class AnkrChainList(BasePage):
    def url(self):
        return "https://www.ankr.com/web3-api/chains-list/"

    def components(self):
        self.card_section = self.page.locator(selector=".jss344")
        self.requests_info_field = self.card_section.locator(".jss539")
        
    def convert_to_number(self, value: str) -> float:
        try:
            if 'B' in value:
                convert_result = float(value.replace('B', '').strip()) * 1e9
                return int(convert_result)
            if 'M' in value:
                convert_result = float(value.replace('M', '').strip()) * 1e6
                return int(convert_result)
            if 'K' in value:
                convert_result = float(value.replace('K', '').strip()) * 1e3
                return int(convert_result)
            return int(value)
        except:
            return None

    def format_large_number(self, number: float) -> str:
        if number >= 1e9:
            return f"{number / 1e9:.1f}B"
        elif number >= 1e6:
            return f"{number / 1e6:.1f}M"
        elif number >= 1e3:
            return f"{number / 1e3:.1f}K"
        else:
            return str(number)

In [127]:
page = await load_default_page()
ankr = AnkrChainList(page)

In [128]:
await ankr.goto()

In [129]:
section = ankr.card_section

In [130]:
requests_info_field = section.locator(".jss539")

In [131]:
total = 0

for idx in range(await requests_info_field.count()):
    target = requests_info_field.nth(idx)
    amount_string = await target.text_content()
    amount_number = ankr.convert_to_number(amount_string)
    if amount_number is not None:
        total += amount_number

In [132]:
ankr.format_large_number(total)

'68.9B'